# Memory-Based Collaborative Filtering (Production-Minded Baseline)

This notebook builds an **item–item collaborative filtering** baseline from a MovieLens-style `ratings.csv`.

**What this notebook demonstrates (for a production-minded review):**
- A clear **data model** (events → sparse matrix)
- A scalable approach: **Top-K neighbors** via cosine similarity (**no dense N×N similarity matrices**)
- A minimal **offline evaluation** (time-aware leave-last-out)
- Reproducibility: configuration, seeding, and artifact outputs

> ⚠️ Compute note: similarity over all items/users can be expensive. This notebook defaults to sampling to keep runtime reasonable.


In [ ]:
# --- Configuration (edit here) ---
import os
from dataclasses import dataclass

@dataclass(frozen=True)
class Config:
    # Data: local path or gs://bucket/path/ratings.csv
    data_path: str = os.environ.get("DATA_PATH", "gs://butterstick2023/ml-25m/ratings.csv")

    # Sampling to keep the notebook fast (set to None to disable)
    max_users: int | None = int(os.environ.get("MAX_USERS", "5000"))
    max_items: int | None = int(os.environ.get("MAX_ITEMS", "20000"))

    # Similarity / retrieval
    top_k: int = int(os.environ.get("TOP_K", "50"))

    # Outputs
    out_dir: str = os.environ.get("OUT_DIR", "./artifacts")

    # Randomness / reproducibility
    seed: int = int(os.environ.get("SEED", "42"))

cfg = Config()
cfg


In [ ]:
# --- Imports ---
import json
import random
from typing import Tuple

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.neighbors import NearestNeighbors


## Data Import

Expected schema (MovieLens-style):

- `userId` (int)
- `movieId` (int)
- `rating` (float)
- `timestamp` (int, unix seconds)


In [ ]:
def _read_csv_any(path: str) -> pd.DataFrame:
    """Read a CSV from local disk or GCS (gs://...).

    Notes:
    - For `gs://...`, you need either:
      - `gcsfs` installed (pandas can read gs:// directly), OR
      - `google-cloud-storage` to download the object first.
    """
    if path.startswith("gs://"):
        try:
            # If gcsfs is available, this just works.
            return pd.read_csv(path)
        except Exception as e:
            # Fallback: download via GCS client.
            from google.cloud import storage  # optional dependency
            import tempfile

            bucket_name, blob_path = path[5:].split("/", 1)
            client = storage.Client()
            bucket = client.bucket(bucket_name)
            blob = bucket.blob(blob_path)

            with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp:
                blob.download_to_filename(tmp.name)
                return pd.read_csv(tmp.name)
    return pd.read_csv(path)


def load_ratings(path: str) -> pd.DataFrame:
    df = _read_csv_any(path)

    required = {"userId", "movieId", "rating", "timestamp"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {sorted(missing)}")

    df = df[list(required)].copy()
    df["userId"] = df["userId"].astype("int64")
    df["movieId"] = df["movieId"].astype("int64")
    df["rating"] = df["rating"].astype("float32")
    df["timestamp"] = df["timestamp"].astype("int64")
    df["event_ts"] = pd.to_datetime(df["timestamp"], unit="s", utc=True)

    return df


df = load_ratings(cfg.data_path)
df.head()


In [ ]:
# Basic sanity checks
df.agg(
    n_rows=("rating", "size"),
    n_users=("userId", "nunique"),
    n_items=("movieId", "nunique"),
    min_ts=("event_ts", "min"),
    max_ts=("event_ts", "max"),
)


## Sampling (to keep runtime reasonable)

This notebook defaults to a user/item cap for review-time execution.
For production, you would run this as a batch job (Spark/BigQuery/Ray) and write versioned artifacts.


In [ ]:
def sample_interactions(
    df: pd.DataFrame,
    max_users: int | None,
    max_items: int | None,
    seed: int,
) -> pd.DataFrame:
    rng = np.random.default_rng(seed)

    if max_users is not None and df["userId"].nunique() > max_users:
        users = rng.choice(df["userId"].unique(), size=max_users, replace=False)
        df = df[df["userId"].isin(users)]

    if max_items is not None and df["movieId"].nunique() > max_items:
        items = rng.choice(df["movieId"].unique(), size=max_items, replace=False)
        df = df[df["movieId"].isin(items)]

    return df


df_s = sample_interactions(df, cfg.max_users, cfg.max_items, cfg.seed).copy()
df_s.agg(n_rows=("rating","size"), n_users=("userId","nunique"), n_items=("movieId","nunique"))


## Feature Engineering

Two simple, explainable features:
- `avg_rating_user`: mean rating per user
- `liked`: rating > user mean (implicit feedback proxy)
- `wt_rating`: rating normalized by user mean


In [ ]:
# Reproducibility
random.seed(cfg.seed)
np.random.seed(cfg.seed)

user_mean = df_s.groupby("userId")["rating"].mean().rename("avg_rating_user")
df_s = df_s.join(user_mean, on="userId")
df_s["liked"] = df_s["rating"] > df_s["avg_rating_user"]
df_s["wt_rating"] = (df_s["rating"] / df_s["avg_rating_user"]).astype("float32")

df_s[["userId","movieId","rating","avg_rating_user","liked","wt_rating"]].head()


## Train/Test Split (time-aware)

For each user, hold out their last interaction as a simple offline eval target.
This prevents leakage from “future” interactions.


In [ ]:
def leave_last_out(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df = df.sort_values(["userId", "event_ts"])
    idx_last = df.groupby("userId").tail(1).index
    test = df.loc[idx_last].copy()
    train = df.drop(idx_last).copy()
    return train, test

train_df, test_df = leave_last_out(df_s)

train_df.shape, test_df.shape


## Build Sparse User–Item Matrix (CSR)

Why CSR:
- Dense pivots explode memory for large U×I
- CSR supports fast linear algebra and nearest-neighbor retrieval


In [ ]:
def build_user_item_csr(
    df: pd.DataFrame,
    user_col: str = "userId",
    item_col: str = "movieId",
    value_col: str = "wt_rating",
) -> Tuple[sparse.csr_matrix, np.ndarray, np.ndarray]:
    # Map ids → contiguous indices
    user_ids, user_idx = np.unique(df[user_col].to_numpy(), return_inverse=True)
    item_ids, item_idx = np.unique(df[item_col].to_numpy(), return_inverse=True)

    data = df[value_col].to_numpy(dtype=np.float32)
    mat = sparse.csr_matrix((data, (user_idx, item_idx)), shape=(len(user_ids), len(item_ids)))
    return mat, user_ids, item_ids

X_train, user_ids, item_ids = build_user_item_csr(train_df)
X_train.shape, X_train.nnz


## Item–Item Top-K Similarity (production-friendly)

Instead of computing a dense item×item similarity matrix, we retrieve **Top-K neighbors** per item.

Why:
- Dense similarity is O(I²) memory
- For serving, you only need top neighbors for candidate generation


In [ ]:
# Fit nearest-neighbors on item vectors (columns) -> use X_train.T (items x users)
nn = NearestNeighbors(
    n_neighbors=cfg.top_k + 1,  # +1 includes the item itself at distance 0
    metric="cosine",
    algorithm="brute",
    n_jobs=-1,
)
nn.fit(X_train.T)

distances, indices = nn.kneighbors(X_train.T, return_distance=True)
# Convert cosine distance to cosine similarity
similarities = 1.0 - distances

# Drop self-match at position 0
neighbor_indices = indices[:, 1:]
neighbor_sims = similarities[:, 1:]

neighbor_indices.shape, neighbor_sims.shape


## Recommendation Functions

Two practical entry points:
1. `similar_items(movie_id)` — item-to-item lookup (great for explainability)
2. `recommend_for_user(user_id)` — aggregate neighbors of items the user liked


In [ ]:
# Build fast lookup: item_id -> row index
item_id_to_row = {int(mid): i for i, mid in enumerate(item_ids)}

def similar_items(movie_id: int, k: int = 10) -> pd.DataFrame:
    if movie_id not in item_id_to_row:
        raise KeyError(f"movieId={movie_id} not in training set (sampled).")

    row = item_id_to_row[movie_id]
    nbr_rows = neighbor_indices[row, :k]
    sims = neighbor_sims[row, :k]
    return pd.DataFrame({
        "movieId": item_ids[nbr_rows].astype(int),
        "similarity": sims.astype(float),
    }).sort_values("similarity", ascending=False)

similar_items(movie_id=int(item_ids[0]), k=10).head()


In [ ]:
# Precompute user->row mapping for the sampled training set
user_id_to_row = {int(uid): i for i, uid in enumerate(user_ids)}

def recommend_for_user(user_id: int, k: int = 10, liked_only: bool = True) -> pd.DataFrame:
    if user_id not in user_id_to_row:
        raise KeyError(f"userId={user_id} not in training set (sampled).")

    urow = user_id_to_row[user_id]
    user_vec = X_train[urow]

    # Items the user interacted with (non-zero entries)
    seen_item_rows = set(user_vec.indices.tolist())

    # Optionally restrict to implicit positives (liked); otherwise use all interactions.
    if liked_only:
        # Identify liked items for this user from the training dataframe
        liked_items = train_df[(train_df["userId"] == user_id) & (train_df["liked"])].movieId.unique()
        liked_rows = [item_id_to_row[int(mid)] for mid in liked_items if int(mid) in item_id_to_row]
        seed_rows = liked_rows if liked_rows else list(seen_item_rows)
    else:
        seed_rows = list(seen_item_rows)

    # Aggregate neighbor scores
    scores = {}
    for r in seed_rows:
        for nbr_r, sim in zip(neighbor_indices[r], neighbor_sims[r]):
            if nbr_r in seen_item_rows:  # don't recommend already-seen items
                continue
            scores[nbr_r] = max(scores.get(nbr_r, 0.0), float(sim))  # max-sim aggregator (simple baseline)

    top = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]
    return pd.DataFrame({
        "movieId": [int(item_ids[r]) for r, _ in top],
        "score": [s for _, s in top],
    })

# Example
example_user = int(user_ids[0])
recommend_for_user(example_user, k=10).head()


## Offline Evaluation (HitRate@K)

We evaluate whether the held-out last item for each user appears in the Top-K recommendations.

This is intentionally lightweight but demonstrates:
- leakage-aware splitting
- a concrete metric tied to ranking quality


In [ ]:
def hitrate_at_k(k: int = 10) -> float:
    hits = 0
    total = 0
    for uid, grp in test_df.groupby("userId"):
        uid = int(uid)
        # If user not in training after sampling/split, skip.
        if uid not in user_id_to_row:
            continue
        held_out = int(grp.iloc[0]["movieId"])
        # If held-out item not in training item universe, skip (sampled out)
        if held_out not in item_id_to_row:
            continue

        recs = recommend_for_user(uid, k=k)
        if held_out in set(recs["movieId"].tolist()):
            hits += 1
        total += 1
    return hits / total if total else float("nan")

for k in [5, 10, 20, 50]:
    print(f"HitRate@{k}: {hitrate_at_k(k):.4f}")


## Persist Artifacts

Artifacts are written locally to `cfg.out_dir`:
- `user_item_matrix_train.npz` (CSR sparse)
- `item_neighbors.parquet` (top-k neighbors per item)

Optional: upload these to object storage (GCS/S3) in a versioned path.


In [ ]:
import os
os.makedirs(cfg.out_dir, exist_ok=True)

# 1) Sparse matrix
sparse.save_npz(os.path.join(cfg.out_dir, "user_item_matrix_train.npz"), X_train)

# 2) Neighbors table
rows = []
for i, mid in enumerate(item_ids):
    for nbr_r, sim in zip(neighbor_indices[i], neighbor_sims[i]):
        rows.append((int(mid), int(item_ids[nbr_r]), float(sim)))

neighbors_df = pd.DataFrame(rows, columns=["movieId", "neighborMovieId", "similarity"])
neighbors_path = os.path.join(cfg.out_dir, "item_neighbors.parquet")
neighbors_df.to_parquet(neighbors_path, index=False)

neighbors_df.head(), neighbors_path
